In [ ]:
!gdown https://drive.google.com/uc?id=1377GxX9ZupjbUgYTjTEzzqPFsapAHNg_

Downloading...
From: https://drive.google.com/uc?id=1377GxX9ZupjbUgYTjTEzzqPFsapAHNg_
To: /content/major.zip
100% 10.1G/10.1G [00:50<00:00, 199MB/s]


In [ ]:
!wget --no-check-certificate data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth

--2022-08-22 15:36:10--  http://data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth
Resolving data.lip6.fr (data.lip6.fr)... 132.227.201.10
Connecting to data.lip6.fr (data.lip6.fr)|132.227.201.10|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth [following]
--2022-08-22 15:36:10--  https://data.lip6.fr/cadene/pretrainedmodels/xception-b5690688.pth
Connecting to data.lip6.fr (data.lip6.fr)|132.227.201.10|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 91674713 (87M) [application/octet-stream]
Saving to: ‘xception-b5690688.pth’

xception-b5690688.p 100%[===================>]  87.43M   363KB/s    in 4m 17s  

2022-08-22 15:40:28 (348 KB/s) - ‘xception-b5690688.pth’ saved [91674713/91674713]



In [ ]:
!mkdir checkpoints
!mkdir checkpoints/F3Net
!mv xception-b5690688.pth checkpoints/F3Net

In [ ]:
!unzip -q major.zip

In [ ]:
!rm major.zip

In [ ]:
!pip install neptune-client torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.0 MB/s 
     |████████████████████████████████| 419 kB 53.2 MB/s 
     |████████████████████████████████| 829 kB 45.5 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 181 kB 71.9 MB/s 
     |████████████████████████████████| 132 kB 65.1 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 9.1 MB 56.5 MB/s 
     |████████████████████████████████| 79 kB 9.5 MB/s 
     |████████████████████████████████| 140 kB 73.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 127 kB 70.8 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 130 kB 74.7 MB/s 
     |████████████████████████████████| 82 kB 813 kB/s 
     |████████████████████████████████| 81 k

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
osenvs = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
from torchvision import transforms as trans
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Recall, Precision, F1Score, Accuracy

import copy

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import RandomCrop

from sklearn.metrics import accuracy_score, roc_curve
from sklearn.metrics import auc as cal_auc

import random

import numpy as np

from PIL import Image

import logging
import sys

import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt 

import neptune.new as neptune
run = neptune.init(
    project="Botz/VIPCup-logs",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkNWJjMDdhNC05NWY5LTQwNWQtYTQyNi0zNjNmYmYwZDg3M2YifQ==",
)

https://app.neptune.ai/Botz/VIPCup-logs/e/VIP-32
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
with open("Major/train.txt","r+") as f0:
  train_list = f0.read().split("\n")

with open("Major/val.txt","r+") as f0:
  val_list = f0.read().split("\n")

with open("Major/test.txt","r+") as f0:
  test_list = f0.read().split("\n")

labels = pd.read_csv("Major/labels.csv")

In [ ]:
def construct_dataloder(dataset, batch_size, shuffle=True, num_workers=4):
    return DataLoader(dataset, batch_size=batch_size,
                      shuffle=shuffle, num_workers=num_workers)

class VIPDataset(Dataset):
    def __init__(self, file_ids, labels, data_dir,):
        self.file_ids = file_ids
        self.labels = labels
        self.data_dir = data_dir
        self.aug = transform = A.Compose([
            A.augmentations.geometric.resize.RandomScale(scale_limit=[0,2],interpolation=cv.INTER_LINEAR,p=0.3),
            A.augmentations.transforms.ImageCompression(quality_lower=99, quality_upper=100, always_apply=False, p=0.3),
            # A.augmentations.crops.transforms.RandomCrop(width=200, height=200,p=1)
        ])
        self.trans = A.Compose([RandomCrop(width=200, height=200,p=1), ToTensorV2()])

    def __len__(self):
        return len(self.file_ids)

    def __getitem__(self, index):
        image_id = self.file_ids[index]
        x = np.array(Image.open(os.path.join(self.data_dir, image_id)))
        x = self.aug(image=x)['image']
        x = self.trans(image=x)['image']
        y = int(self.labels.loc[self.labels["image_ids"] == image_id,"label"])
        x = 2*x.float()/255.0 - 1
        return x, torch.from_numpy(np.array(y))

    def augment(self, x):
        """Augmentations for images"""
        return self.aug_pipeline(image=x)["image"]

In [ ]:
train_ds = VIPDataset(train_list,labels,"Major/data")
val_ds = VIPDataset(val_list,labels,"Major/data")
test_ds = VIPDataset(test_list,labels,"Major/data")

In [ ]:
batch_size = 8

train_dl = construct_dataloder(train_ds, batch_size, shuffle=True, num_workers=2)
val_dl = construct_dataloder(val_ds, batch_size, shuffle=True, num_workers=2)
test_dl = construct_dataloder(test_ds, batch_size, shuffle=True, num_workers=2)

In [ ]:
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x

class Block(nn.Module):
    def __init__(self,in_filters,out_filters,reps,strides=1,start_with_relu=True,grow_first=True):
        super(Block, self).__init__()

        if out_filters != in_filters or strides!=1:
            self.skip = nn.Conv2d(in_filters,out_filters,1,stride=strides, bias=False)
            self.skipbn = nn.BatchNorm2d(out_filters)
        else:
            self.skip=None

        self.relu = nn.ReLU(inplace=True)
        rep=[]

        filters=in_filters
        if grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))
            filters = out_filters

        for i in range(reps-1):
            rep.append(self.relu)
            rep.append(SeparableConv2d(filters,filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(filters))

        if not grow_first:
            rep.append(self.relu)
            rep.append(SeparableConv2d(in_filters,out_filters,3,stride=1,padding=1,bias=False))
            rep.append(nn.BatchNorm2d(out_filters))

        if not start_with_relu:
            rep = rep[1:]
        else:
            rep[0] = nn.ReLU(inplace=False)

        if strides != 1:
            rep.append(nn.MaxPool2d(3,strides,1))
        self.rep = nn.Sequential(*rep)

    def forward(self,inp):
        x = self.rep(inp)

        if self.skip is not None:
            skip = self.skip(inp)
            skip = self.skipbn(skip)
        else:
            skip = inp

        x+=skip
        return x

# MISSING 'ClassBlock <- classify the final result'

class Xception(nn.Module):
    """
    Xception optimized for the ImageNet dataset, as specified in
    https://arxiv.org/pdf/1610.02357.pdf
    """
    def __init__(self, num_classes=1000):
        """ Constructor
        Args:
            num_classes: number of classes
        """
        super(Xception, self).__init__()
        self.num_classes = num_classes

        self.conv1 = nn.Conv2d(3, 32, 3,2, 0, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(32,64,3,bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        #do relu here

        self.block1=Block(64,128,2,2,start_with_relu=False,grow_first=True)
        self.block2=Block(128,256,2,2,start_with_relu=True,grow_first=True)
        self.block3=Block(256,728,2,2,start_with_relu=True,grow_first=True)

        self.block4=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block5=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block6=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block7=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block8=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block9=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block10=Block(728,728,3,1,start_with_relu=True,grow_first=True)
        self.block11=Block(728,728,3,1,start_with_relu=True,grow_first=True)

        self.block12=Block(728,1024,2,2,start_with_relu=True,grow_first=False)

        self.conv3 = SeparableConv2d(1024,1536,3,1,1)
        self.bn3 = nn.BatchNorm2d(1536)

        #do relu here
        self.conv4 = SeparableConv2d(1536,2048,3,1,1)
        self.bn4 = nn.BatchNorm2d(2048)

        # Missing 'fc <- Linear(2048, 1000)' and 'dp <- Dropout(p=0.2)' layers

        # Missing 'forward' function

def return_pytorch04_xception(pretrained=True):
    model = Xception(num_classes=1000)
    if pretrained:
        state_dict = torch.load(
            'checkpoints/F3Net/xception-b5690688.pth')
        for name, weights in state_dict.items():
            if 'pointwise' in name:
                state_dict[name] = weights.unsqueeze(-1).unsqueeze(-1)
        model.load_state_dict(state_dict, strict=False)
 
    return model

In [ ]:
# utils
def DCT_mat(size):
    m = [[ (np.sqrt(1./size) if i == 0 else np.sqrt(2./size)) * np.cos((j + 0.5) * np.pi * i / size) for j in range(size)] for i in range(size)]
    return m

def generate_filter(start, end, size):
    return [[0. if i + j > end or i + j <= start else 1. for j in range(size)] for i in range(size)]

def norm_sigma(x):
    return 2. * torch.sigmoid(x) - 1.

class Filter(nn.Module):
    def __init__(self, size, 
                 band_start, 
                 band_end, 
                 use_learnable=True, 
                 norm=False):
        super(Filter, self).__init__()
        self.use_learnable = use_learnable

        self.base = nn.Parameter(torch.tensor(generate_filter(band_start, band_end, size)), requires_grad=False)
        if self.use_learnable:
            self.learnable = nn.Parameter(torch.randn(size, size), requires_grad=True)
            self.learnable.data.normal_(0., 0.1)
            # REPEATING OPERATIONS?
            # Todo
            # self.learnable = nn.Parameter(torch.rand((size, size)) * 0.2 - 0.1, requires_grad=True)

        self.norm = norm
        if norm:
            self.ft_num = nn.Parameter(torch.sum(torch.tensor(generate_filter(band_start, band_end, size))), requires_grad=False)


    def forward(self, x):
        if self.use_learnable:
            filt = self.base + norm_sigma(self.learnable)
        else:
            filt = self.base

        if self.norm:
            y = x * filt / self.ft_num
        else:
            y = x * filt
        return y

class FAD_Head(nn.Module):
    def __init__(self, size):
        super(FAD_Head, self).__init__()
        # init DCT matrix
        self._DCT_all = nn.Parameter(torch.tensor(DCT_mat(size)).float(), requires_grad=False)
        self._DCT_all_T = nn.Parameter(torch.transpose(torch.tensor(DCT_mat(size)).float(), 0, 1), requires_grad=False)

        # define base filters and learnable
        # 0 - 1/16 || 1/16 - 1/8 || 1/8 - 1
        low_filter = Filter(size, 0, size // 16)
        middle_filter = Filter(size, size // 16, size // 8)
        high_filter = Filter(size, size // 8, size)
        all_filter = Filter(size, 0, size * 2)
        # BANDS ARE DIFFERENT

        self.filters = nn.ModuleList([low_filter, middle_filter, high_filter, all_filter])

    def forward(self, x):
        # DCT
        x_freq = self._DCT_all @ x @ self._DCT_all_T    # [N, 3, 200, 200]

        # 4 kernel
        y_list = []
        for i in range(4):
            x_pass = self.filters[i](x_freq)  # [N, 3, 200, 200]
            y = self._DCT_all_T @ x_pass @ self._DCT_all    # [N, 3, 200, 200]
            y_list.append(y)
        out = torch.cat(y_list, dim=1)    # [N, 12, 200, 200]
        return out

class LFS_Head(nn.Module):
    def __init__(self, size, window_size, M):
        super(LFS_Head, self).__init__()

        self.window_size = window_size
        self._M = M

        # init DCT matrix
        self._DCT_patch = nn.Parameter(torch.tensor(DCT_mat(window_size)).float(), requires_grad=False)
        self._DCT_patch_T = nn.Parameter(torch.transpose(torch.tensor(DCT_mat(window_size)).float(), 0, 1), requires_grad=False)

        self.unfold = nn.Unfold(kernel_size=(window_size, window_size), stride=2, padding=4)

        # init filters
        self.filters = nn.ModuleList([Filter(window_size, window_size * 2. / M * i, window_size * 2. / M * (i+1), norm=True) for i in range(M)])
    
    def forward(self, x):
        # turn RGB into Gray
        x_gray = 0.299*x[:,0,:,:] + 0.587*x[:,1,:,:] + 0.114*x[:,2,:,:]
        x = x_gray.unsqueeze(1)

        # rescale to 0 - 255
        x = (x + 1.) * 122.5

        # calculate size
        N, C, W, H = x.size()
        S = self.window_size
        size_after = int((W - S + 8)/2) + 1
        assert size_after == 100

        # sliding window unfold and DCT
        x_unfold = self.unfold(x)   # [N, C * S * S, L]   L:block num
        L = x_unfold.size()[2]
        x_unfold = x_unfold.transpose(1, 2).reshape(N, L, C, S, S)  # [N, L, C, S, S]
        x_dct = self._DCT_patch @ x_unfold @ self._DCT_patch_T

        # M kernels filtering
        y_list = []
        for i in range(self._M):
            # y = self.filters[i](x_dct)    # [N, L, C, S, S]
            # y = torch.abs(y)
            # y = torch.sum(y, dim=[2,3,4])   # [N, L]
            # y = torch.log10(y + 1e-15)
            y = torch.abs(x_dct)
            y = torch.log10(y + 1e-15)
            y = self.filters[i](y)
            y = torch.sum(y, dim=[2,3,4])
            y = y.reshape(N, size_after, size_after).unsqueeze(dim=1)   # [N, 1, 100, 100]
            y_list.append(y)
        out = torch.cat(y_list, dim=1)  # [N, M, 100, 100]
        return out

class MixBlock(nn.Module):
    
    def __init__(self, c_in, width, height):
        super(MixBlock, self).__init__()
        self.FAD_query = nn.Conv2d(c_in, c_in, (1,1))
        self.LFS_query = nn.Conv2d(c_in, c_in, (1,1))

        self.FAD_key = nn.Conv2d(c_in, c_in, (1,1))
        self.LFS_key = nn.Conv2d(c_in, c_in, (1,1))

        self.softmax = nn.Softmax(dim=-1)
        self.relu = nn.ReLU()

        self.FAD_gamma = nn.Parameter(torch.zeros(1))
        self.LFS_gamma = nn.Parameter(torch.zeros(1))

        self.FAD_conv = nn.Conv2d(c_in, c_in, (1,1), groups=c_in)
        self.FAD_bn = nn.BatchNorm2d(c_in)
        self.LFS_conv = nn.Conv2d(c_in, c_in, (1,1), groups=c_in)
        self.LFS_bn = nn.BatchNorm2d(c_in)

    def forward(self, x_FAD, x_LFS):
        B, C, W, H = x_FAD.size()
        assert W == H

        q_FAD = self.FAD_query(x_FAD).view(-1, W, H)    # [BC, W, H]
        q_LFS = self.LFS_query(x_LFS).view(-1, W, H)
        M_query = torch.cat([q_FAD, q_LFS], dim=2)  # [BC, W, 2H]

        k_FAD = self.FAD_key(x_FAD).view(-1, W, H).transpose(1, 2)  # [BC, H, W]
        k_LFS = self.LFS_key(x_LFS).view(-1, W, H).transpose(1, 2)
        M_key = torch.cat([k_FAD, k_LFS], dim=1)    # [BC, 2H, W]

        energy = torch.bmm(M_query, M_key)  #[BC, W, W]
        attention = self.softmax(energy).view(B, C, W, W)

        att_LFS = x_LFS * attention * (torch.sigmoid(self.LFS_gamma) * 2.0 - 1.0)
        y_FAD = x_FAD + self.FAD_bn(self.FAD_conv(att_LFS))

        att_FAD = x_FAD * attention * (torch.sigmoid(self.FAD_gamma) * 2.0 - 1.0)
        y_LFS = x_LFS + self.LFS_bn(self.LFS_conv(att_FAD))
        return y_FAD, y_LFS

class F3Net(nn.Module):
    """
    Implementation is mainly referenced from https://github.com/yyk-wew/F3Net
    """
    def __init__(self, 
                 num_classes: int=2, 
                 img_width: int=200, 
                 img_height: int=200, 
                 LFS_window_size: int=10, 
                 LFS_M: int=6) -> None:
        super(F3Net, self).__init__()
        assert img_width == img_height
        self.img_size = img_width
        self.num_classes = num_classes
        self._LFS_window_size = LFS_window_size
        self._LFS_M = LFS_M
        
        
        self.fad_head = FAD_Head(self.img_size)
        self.lfs_head = LFS_Head(self.img_size, self._LFS_window_size, self._LFS_M)
        
        self.fad_excep = self._init_xcep_fad()
        self.lfs_excep = self._init_xcep_lfs()
        
        self.mix_block7 = MixBlock(c_in=728, width=19, height=19) 
        self.mix_block12 = MixBlock(c_in=1024, width=10, height=10) 
        self.excep_forwards = ['conv1', 'bn1', 'relu', 'conv2', 'bn2', 'relu', 
                               'block1', 'block2', 'block3', 'block4', 'block5', 'block6', 
                               'block7', 'block8', 'block9', 'block10' , 'block11', 'block12',
                               'conv3', 'bn3', 'relu', 'conv4', 'bn4']

         # classifier
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(4096, num_classes)
        self.dp = nn.Dropout(p=0.2)
        
    def _init_xcep_fad(self):
        fad_excep =  return_pytorch04_xception(True)
        conv1_data = fad_excep.conv1.weight.data
        # let new conv1 use old param to balance the network
        fad_excep.conv1 = nn.Conv2d(12, 32, 3, 2, 0, bias=False)
        for i in range(4):
            fad_excep.conv1.weight.data[:, i*3:(i+1)*3, :, :] = conv1_data / 4.0
        return fad_excep
    
    def  _init_xcep_lfs(self): 
        lfs_excep = return_pytorch04_xception(True)
        conv1_data = lfs_excep.conv1.weight.data
        # let new conv1 use old param to balance the network
        lfs_excep.conv1 = nn.Conv2d(self._LFS_M, 32, 3, 1, 1, bias=False)
        for i in range(int(self._LFS_M / 3)):
            lfs_excep.conv1.weight.data[:, i*3:(i+1)*3, :, :] = conv1_data / float(self._LFS_M / 3.0)
        return lfs_excep
    
    def _features(self, x_fad, x_fls):
        for forward_func in self.excep_forwards:
            x_fad = getattr(self.fad_excep, forward_func)(x_fad)
            x_fls = getattr(self.lfs_excep, forward_func)(x_fls)
            if forward_func == 'block7':
                x_fad, x_fls = self.mix_block7(x_fad, x_fls)
            if forward_func == 'block12':
                x_fad, x_fls = self.mix_block12(x_fad, x_fls)
        return x_fad, x_fls
    
    def _norm_feature(self, x):
        x = self.relu(x)
        x = F.adaptive_avg_pool2d(x, (1,1))
        x = x.view(x.size(0), -1)
        return x
    
    def forward(self, x):
        fad_input = self.fad_head(x)
        lfs_input = self.lfs_head(x)
        x_fad, x_fls = self._features(fad_input, lfs_input)
        x_fad = self._norm_feature(x_fad)
        x_fls = self._norm_feature(x_fls)
        x_cat = torch.cat((x_fad, x_fls), dim=1)
        x_drop = self.dp(x_cat)
        logit = self.fc(x_drop)
        return x_cat, logit

In [ ]:
def evaluate(model, mode='valid', recorder=None):
    if mode=='valid': dl = val_dl
    elif mode=='test': dl = test_dl
    else:
        print('Unknown mode')
        return 0,0,0

    with torch.no_grad():
        y_true, y_pred = [], []

        for i, (data,label) in enumerate(dl):
            output = model.forward(data)
            y_pred.extend(torch.argmax(output.sigmoid(), dim=-1).flatten().tolist())
            y_true.extend(label.flatten().tolist())
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true, y_pred = torch.tensor(y_true), torch.tensor(y_pred)

    idx_real = np.where(y_true==0)[0]
    idx_fake = np.where(y_true==1)[0]

    acc = accuracy_score(y_true, y_pred > 0.5)
    r_acc = accuracy_score(y_true[idx_real], y_pred[idx_real] > 0.5)
    f_acc = accuracy_score(y_true[idx_fake], y_pred[idx_fake] > 0.5)

    for i in record_list:
        recorder[i].append(metrics[i](y_pred, y_true))

In [ ]:
def initModel(mod, gpu_ids):
    mod = mod.to(f'cuda:{gpu_ids[0]}')
    mod = nn.DataParallel(mod, gpu_ids)
    return mod

class Trainer(): 
    def __init__(self, gpu_ids):
        self.model = F3Net()
        self.model = initModel(self.model, gpu_ids)
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, self.model.parameters()),
                                              lr=0.0002, betas=(0.9, 0.999))
        self.total_steps = 0
        # self.optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, self.model.parameters()),
        #                                         lr=0.002, momentum=0.9, weight_decay=0)
    
    def __call__(self, data, label):
        stu_fea, stu_cla = self.model(data)

        self.loss_cla = self.loss_fn(stu_cla.squeeze(1), label) # classify loss
        self.loss = self.loss_cla

        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        return self.loss
    
    def forward(self, data):
        stu_fea, stu_cla = self.model(data)
        return stu_cla

    def save(self, path):
        torch.save(self.model.state_dict(), path)

    def load(self, path):
        state_dict = torch.load(path)
        self.model.load_state_dict(state_dict)

In [ ]:
gpu_ids = [*range(osenvs)]
epochs = 10
loss_freq = 40
val_freq = 1
device = torch.device('cuda:{}'.format(gpu_ids[0])) if gpu_ids else torch.device('cpu')
metrics = {
    'accuracy': Accuracy(num_classes=2, average='macro'),
    'recall': Recall(num_classes=2, average='macro'),
    'precision': Precision(num_classes=2, average='macro'),
    'f1_score': F1Score(num_classes=2, average='macro')
}

params = {
    "Model-Name": 'Thinking in Frequency',
    'Dataset-Name': 'Combined datasetv1',
    "backbone_model": "xception",
    "output_type": "none",
    "batch_size": batch_size,
    "learning_rate": 0.0002, 
    "optimizer": "Adam",
    "use_reg": False
}
run["parameters"] = params

if __name__ == '__main__':   
    # train
    model = Trainer(gpu_ids)
    best_acc = 0
    best_model_wts = copy.deepcopy(model.model.state_dict())

    record_list = list(metrics.keys())
    
    for epoch in range(epochs):   
        recorder = {metric: [] for metric in record_list}    
        # logger.debug(f'No {epoch}')
        print(f'Epoch {epoch+1}')
        for i,(data,label) in enumerate(train_dl):
            model.total_steps += 1

            label = torch.nn.functional.one_hot(label, num_classes=2).float()
            data, label = data.to(device), label.to(device)

            loss = model(data,label)

            if model.total_steps % loss_freq == 0:
                print(f'loss: {loss} at step: {model.total_steps}')

        model.model.eval()
        evaluate(model, mode='valid', recorder=recorder)
        for i in record_list:
            print(f'{i}: {100*sum(recorder[i])/len(recorder[i]):.4f}')
        for i in record_list:
            run[f'val/{i}'].log(sum(recorder[i])/len(recorder[i]))
        acc = sum(recorder['accuracy'])/len(recorder['accuracy'])
        if acc > best_acc:
            best_acc = acc
            best_model_wts = copy.deepcopy(model.model.state_dict())
        model.model.train()

    model.model.load_state_dict(best_model_wts)

    model.model.eval()
    print('Testing')
    recorder = {metric: [] for metric in record_list}
    evaluate(model, mode='test', recorder=recorder)
    for i in record_list:
        print(f'{i}: {100*sum(recorder[i])/len(recorder[i]):.4f}')
    for i in record_list:
        run[f'eval/{i}'].log(sum(recorder[i])/len(recorder[i]))

Epoch 1
loss: 0.5752686262130737 at step: 40
loss: 0.6340379118919373 at step: 80
loss: 0.731475293636322 at step: 120
loss: 0.38900837302207947 at step: 160
loss: 0.414130300283432 at step: 200
loss: 0.46421730518341064 at step: 240
loss: 0.563730001449585 at step: 280
loss: 0.338836133480072 at step: 320
loss: 0.46689528226852417 at step: 360
loss: 0.2763415575027466 at step: 400
loss: 0.4023626446723938 at step: 440
loss: 0.6912767887115479 at step: 480
loss: 0.6350280046463013 at step: 520
loss: 0.46378281712532043 at step: 560
loss: 0.38623175024986267 at step: 600
loss: 0.35194098949432373 at step: 640
loss: 0.3239670991897583 at step: 680
loss: 0.41498130559921265 at step: 720
loss: 0.41055193543434143 at step: 760
loss: 0.7813778519630432 at step: 800
loss: 0.24339568614959717 at step: 840
loss: 0.7975344657897949 at step: 880
loss: 0.2987203299999237 at step: 920
loss: 0.43118417263031006 at step: 960
loss: 0.5094676613807678 at step: 1000
loss: 0.8089606165885925 at step: 104

In [ ]:
exp_num = 'VIP-32'
model.save(f'/content/checkpoints/F3Net/{exp_num}.pth')
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/Botz/VIPCup-logs/e/VIP-32


In [ ]:
from torchsummary import summary
summary(model.model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Filter-1          [-1, 3, 200, 200]               0
            Filter-2          [-1, 3, 200, 200]               0
            Filter-3          [-1, 3, 200, 200]               0
            Filter-4          [-1, 3, 200, 200]               0
          FAD_Head-5         [-1, 12, 200, 200]               0
            Unfold-6           [-1, 100, 10000]               0
            Filter-7     [-1, 10000, 1, 10, 10]               0
            Filter-8     [-1, 10000, 1, 10, 10]               0
            Filter-9     [-1, 10000, 1, 10, 10]               0
           Filter-10     [-1, 10000, 1, 10, 10]               0
           Filter-11     [-1, 10000, 1, 10, 10]               0
           Filter-12     [-1, 10000, 1, 10, 10]               0
         LFS_Head-13          [-1, 6, 100, 100]               0
           Conv2d-14           [-1, 32,